In [1]:
pip install requests 
import requests 

Note: you may need to restart the kernel to use updated packages.


In [3]:
standings_url =  "https://fbref.com/en/comps/9/Premier-League-Stats" 
data = requests.get(standings_url)

In [5]:
pip install bs4

from bs4 import BeautifulSoup
import time
time.sleep(5)
soup = BeautifulSoup(data.text)

Note: you may need to restart the kernel to use updated packages.


In [10]:
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
team_url = team_urls[0]
team_url
data = requests.get(team_url)

In [18]:
pip install pandas
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
from io import StringIO

html_content = StringIO(data.text)
matches = pd.read_html(html_content, match="Scores & Fixtures")[0]

In [21]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
data = requests.get(f"https://fbref.com{links[0]}")

In [26]:

import pandas as pd 
from io import StringIO

html_content = StringIO(data.text)
shooting = pd.read_html(html_content, match="Shooting")[0]

In [27]:
shooting.columns = shooting.columns.droplevel()
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data.head()
years = list(range(2025, 2019, -1))


all_matches = []


In [32]:
%pip install html5lib


Note: you may need to restart the kernel to use updated packages.


In [33]:
import time
from io import StringIO
#For loop that loops through each season starting from
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    time.sleep(5)

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        time.sleep(3)
        try:
            matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        except ValueError:
            continue
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        try:
            shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        except ValueError:
            continue
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(7)
        
       

    



In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv")